In [1]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
from torch.jit.annotations import List
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [2]:
__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']

model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}

In [3]:
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)), # 배치정규화.
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)),

        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)), #bn_size가 bottleNeck의미. 이렇게 곱해준 이유가 좀더 다양하게 처리하려고.
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)),

        self.drop_rate = float(drop_rate) #논문 볼때, 드롭드롭 했던게 이걸 말하는 듯 하다.확률적으로 떨어뜨림.
        self.memory_efficient = memory_efficient #메모리 최적값 찾아서 써야함.(트레이드 오프 생각. -> 연산량이 증가할 수 있다.)

    def bn_function(self, inputs):
        concated_features = torch.cat(inputs, 1) #1차원으로 쫘악 펴준담에
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features))) #이게 병목 계층. 논문 표 참고하자.
        return bottleneck_output

    
    def any_requires_grad(self, input):
        for tensor in input:
            if tensor.requires_grad:
                return True
        return False

    def call_checkpoint_bottleneck(self, input):
        
        def closure(*inputs):
            return self.bn_function(inputs)

        return cp.checkpoint(closure, *input)

    def forward(self, input):

        pass

    def forward(self, input):

        pass

    def forward(self, input): #순전파.
        if isinstance(input, Tensor): # 들어온 놈이 Tensor라면 
            prev_features = [input] #여기로
        else:
            prev_features = input #아님 그대로 .

        if self.memory_efficient and self.any_requires_grad(prev_features): #특징맵들이 기울기 필요한지 확인.
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features) # 기울기 필요하면, 아까 bottleNeck만들어 준거 계산.
        else:
            bottleneck_output = self.bn_function(prev_features) #필요 없음 아까 그 bn_function해준 거로 병목 계산.

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))

        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, #dropout.
                                     training=self.training)
        return new_features

In [4]:
class _DenseBlock(nn.ModuleDict):
    _version = 2 # 그냥 두번째 클래스라는 것. 첨 봄 ㅋ


    #아까 함수 만들어 둔것으로 요리조리.
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

   # 순전파 계산
    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)

In [5]:
class _Transition(nn.Sequential):# 논문에서 보던 전이 전이 하던 것.
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features)) #전이 블록에 배치 정규화.
        self.add_module('relu', nn.ReLU(inplace=True)) #ReLU활성화 함수 추가하고,
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features, # 1x1 Conv 추가.
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2)) # 2x2 avg풀링 추가.

In [6]:
class DenseNet(nn.Module):
   
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000, memory_efficient=False):

        super(DenseNet, self).__init__()

        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, # 7x7 Conv
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)), #배치 정규화. 
            ('relu0', nn.ReLU(inplace=True)), #ReLU
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)), #MaxPool
        ]))

        num_features = num_init_features

        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate

            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, # 아까생성 해줬던 , 전이(transition).
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


def _load_state_dict(model, model_url, progress): #모델 현재 상태 로드(가중치, 파라미터 포함.)

    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

    state_dict = load_state_dict_from_url(model_url, progress=progress)
    for key in list(state_dict.keys()):
        res = pattern.match(key)
        if res:
            new_key = res.group(1) + res.group(2)
            state_dict[new_key] = state_dict[key]
            del state_dict[key]
    model.load_state_dict(state_dict)


def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs) # DenseNet 클래스, 객체 생성.
    
    if pretrained:
        _load_state_dict(model, model_urls[arch], progress)  # 아까 만들어 준 것으로, 가중치 학습된 것 로드.
    return model


def densenet121(pretrained=False, progress=True, **kwargs): # DenseNet-121
   
    return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,   # 32, (6, 12, 24, 16), 64,
                     **kwargs)                                                       # growth_rate, (1st 블록 6개 층, 2nd 블록 12개 층 ...), num_init_features(초기 입력 이미지 채널 수)


def densenet161(pretrained=False, progress=True, **kwargs): # DenseNet-161
    
    return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
                     **kwargs)


def densenet169(pretrained=False, progress=True, **kwargs): #DenseNet-169
    return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
                     **kwargs)


def densenet201(pretrained=False, progress=True, **kwargs): # DenseNet-201
  return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
                     **kwargs)